In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr  6 16:09:25 2018

@author: Shubadra
"""

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import numpy as np
import string
import re
from functools import reduce

find = lambda searchList, elem: [[i for i, x in enumerate(searchList) if x == e] for e in elem]
fname = 'project_2_train/' + 'data 2_train.csv'
# fname = 'project_2_train/' + 'data 2_train.csv'

stopwords_set = set(stopwords.words('english'))
stopwords_set = {'i', 'shan', 'just', 'how', 'each', 'out', 'themselves', 'their', 'before', 'were', 'very', 'as', 'further', 'his', 'a', 'once', 'youve', 'y', 'is', 'shouldve', 'youll', 'on', 'd', 'm', 'under', 'haven', 'which', 'only', 'them', 'was', 'by', 'needn', 'whom', 'that', 'when', 's', 'isn', 'its', 'no', 'wasn', 'in', 'we', 'theirs', 'those', 'this', 'having', 'and', 'ain', 'most', 'up', 'off', 'being', 'aren', 'shouldn', 'ourselves', 'from', 'down', 'herself', 'her', 'you', 'are', 'its', 'who', 'the', 'here', 'where', 'your', 'youd', 'she', 'didn', 'weren', 'about', 'has', 'our', 'an', 'yourselves', 'or', 'hasn', 'again', 'while', 'does', 'him', 'shes', 'above', 'below', 'itself', 'to', 'through', 'will', 'couldn', 'hers', 'they', 'doing', 'because', 'he', 'what', 'such', 'youre', 'nor', 'too', 'should', 'ours', 'then', 'himself', 'all', 'of', 'mightn', 'between', 'now', 'against', 'some', 'with', 'until', 'am', 'other', 'at', 'can', 'over', 'mustn', 'wouldn', 'do', 'for', 'after', 'hadn', 'me', 'been', 'same', 'doesn', 'my', 'these', 'll', 'did', 'had', 'it', 'so', 'ma', 'during', 'than', 'o', 'yourself', 'own', 'have', 're', 've', 'be', 'why', 't', 'there', 'more', 'won', 'yours', 'few', 'into', 'thatll', 'any', 'myself', 'both', 'don', 'if'}
# print(stopwords_set)
# stopwords_set.remove('but')
# stopwords_set.remove('not')

sentence_weights = []
X, Y, X1 = [], [], []

f = open(fname, 'r')
for i, line in enumerate(f):
    if i != 0:
        ''' Splitting the columns based on comma - since it is csv'''
        columns = line.split(',')

        ''' The comma in the actual sentence was represented as [commma] because of csv format, replace that '''
        columns[1] = columns[1].replace('[comma]', ',')
        columns[2] = columns[2].replace('[comma]', ',')

        ''' Some aspect terms are mic in the sentence where computer/mic is present '''
        columns[1] = columns[1].replace('-', ' ')
        columns[2] = columns[2].replace('-', ' ')
        columns[1] = columns[1].replace('/', ' ')
        columns[2] = columns[2].replace('/', ' ')

        ''' NOt used anymore'''
        tokenizer = RegexpTokenizer(r'\w+') # doesn't work if the special char is in the token
        # columns[2] = tokenizer.tokenize(columns[2].lower())

        '''Tokenize the words'''
        columns[1] = word_tokenize(columns[1])
        columns[2] = word_tokenize(columns[2])

        for j, elem in enumerate(columns[1]):
            '''Remove special characters'''
            columns[1][j] = re.sub('[^0-9a-zA-Z]+', '', elem).lower()
            ''' didn't is tokenized as did n't hence following code '''
            if columns[1][j] == 'nt':
                columns[1][j] = 'not'

        for j, elem in enumerate(columns[2]):
            '''Remove special characters'''
            columns[2][j] = re.sub('[^0-9a-zA-Z]+', '', elem).lower()
            ''' didn't is tokenized as did n't hence following code '''
            if columns[2][j] == 'nt':
                columns[2][j] = 'not'

        '''Remove empty string tokens'''
        columns[1] = [x.strip() for x in columns[1] if x.strip() != '']
        columns[2] = [x.strip() for x in columns[2] if x.strip() != '']

        ''' Remove stop words '''
        columns[1] = [word for word in columns[1] if word not in stopwords_set]
        columns[1] = [word for word in columns[1] if word not in stopwords_set]
        columns[2] = [word for word in columns[2] if word not in stopwords_set]

        columns[1] = ' '.join(columns[1])
        columns[2] = ' '.join(columns[2])

        ''' The aspect term location given is not proper - hence extracting the location by ourselves '''
        columns[3] = []
        ''' Finds the location of the aspect term in the sentence '''
        for m in re.finditer(columns[2], columns[1]):
            columns[3].append([m.start(), m.end()])
            ''' since some aspect terms are not surrounded by spaces, hence below code
            NOT USED ANYMORE
            '''
            if m.start() != 0 and columns[1][m.start() - 1] != ' ':
                # columns[1] = columns[1][:m.start()] + ' ' + columns[1][m.start():]
                pass

        ''' Because of adding space in above code, the position is messed up, hence redoing it. '''
        columns[3] = []

        for m in re.finditer(columns[2], columns[1]):
            columns[3].append([m.start(), m.end()])

        ''' Tokenizing the words again '''
        columns[1] = columns[1].split(' ')
        columns[2] = columns[2].split(' ')
        ''' New column to specify the aspect term location in the list '''
        columns.append([])
        # columns[5] = []
        ''' for multiple positions of aspect term '''
        for elem in columns[3]:
            if elem[0] == 0: #start index of the 1st occurance of the aspect term
                columns[5].append([0])
            else:
                temp_len = elem[0] # assign the start position of the aspect term
                for j, tokens in enumerate(columns[1]):
                    temp_len -= (len(tokens) + 1) # Counting the words till the position - +1 for blank space
                    if temp_len == 0: # Reached the aspect word
                        columns[5].append([j+1])
                        break
            for k in range(len(columns[2])-1): # if multiple words in the aspect term - tag the following words in the sentence
                columns[5][-1].append(columns[5][-1][-1]+1)

        ''' if not found the aspect term location in the list then do '''
        if len(columns[5]) < 1:
            continue

            # pass

        word_wt_list = [[elem, 0] for elem in columns[1]]
        ''' Assigning weights to every word based on the distance from the aspect term '''
        for j, elem in enumerate(columns[5]):
            for k, word in enumerate(columns[1]):
                if k < elem[0]: # Word left to the aspect term
                    dist = abs(elem[0] - k)
                    if word_wt_list[k][1] < 1/dist:
                        word_wt_list[k][1] = 1/dist
                elif k > elem[-1]: # word right to the aspect term : Aspect term can have multiple words
                    dist = abs(k - elem[-1])
                    if word_wt_list[k][1] < 1/dist:
                        word_wt_list[k][1] = 1/dist
            for aspect_word_loc in elem:
                word_wt_list[aspect_word_loc][1] = 1.5

        ''' For duplicate words - if it's aspect term - add the weights, if not, then take the weight which is greater, make the other weight as zero '''
        for j, word_1 in enumerate(word_wt_list):
            for k, word_2 in enumerate(word_wt_list[j+1:]):
                if word_1[0] == word_2[0]:
                    if word_1[1] == 1.5 and word_2[1] == 1.5:
                        # print(line)
                        # print(word_wt_list)
                        word_1[1] += 1.5
                        word_2[1] = 0
                    elif word_1[1] < word_2[1]:
                        # print(line)
                        # print(word_wt_list)
                        word_1[1] = word_2[1]
                        word_2[1] = 0
                    elif word_1[1] > word_2[1]:
                        # print(line)
                        # print(word_wt_list)
                        word_2[1] = 0
                    elif word_1[1] == word_2[1]:
                        word_2[1] = 0


        ''' For removing duplicate words - they have weight zero now'''
        word_wt_list = [elem for elem in word_wt_list if elem[1] != 0]

        if len(columns[2]) > 1 and len(columns[5]) > 1:
            # print(line)
            # print(columns)
            # print(word_wt_list)
            pass

        if columns[0] == '2911_0':
            # print(line)
            # print(word_wt_list)
            pass
        if i == 100:
            # break
            pass

        '''Removing new line character from columns[4] -  the polarity column'''
        columns[4] = columns[4].rstrip('\n')
        X.append(columns[1])
        X1.append(' '.join(columns[1]))

        Y.append(columns[4])
        sentence_weights.append(word_wt_list)


'''Building vocabulary and counting word occurances'''
X_reduced = reduce(lambda x1, x2: x1 + x2, X)
vocab = list(set(X_reduced))
print(vocab[0])
weight_v = np.zeros_like(vocab, dtype = np.float_)
weight_x = []
# pf = open('weightxy_data1.csv', 'w')
pf = open('weightxy_data2.csv', 'w')

pf.write(','.join(vocab))
pf.write(',class_')
pf.write('\n')
for i, sentence in enumerate(sentence_weights):
    for word in sentence:
        v_index = vocab.index(word[0])
        weight_v[v_index] = word[1]

    weight_x.append(weight_v)
    if len(sentence) != weight_v[np.where(weight_v > 0)].shape[0]:
        print(sentence)
        print(len(sentence))
        print(weight_v[np.where(weight_v > 0)])

    ''' code to write the data to file '''
    temp = [str(j) for j in weight_v.tolist()]
    pf.write(','.join(temp))
    pf.write(','+str(Y[i]))
    pf.write('\n')
    weight_v = np.zeros_like(weight_v)

pf.close()
weight_x = np.array(weight_x)

print(weight_x.shape)

f.close()



# weight_X2 = np.reshape(weight_X, (len(Y) ,weight_X.shape[0]))
# print(weight_X2.shape)
# tokens = f.split(',') # columns
# remove punctuations
# table = str.maketrans('', '', string.punctuation)
# print(table)
# tokens = [w.translate(table) for w in tokens]
# # remove tokens that are not alphbetic
# tokens = [word for word in tokens if word.isalpha()]
#
# # remove stop words
# stop_words = set(stopwords.words('english'))
# tokens = [w for w in tokens if not w in stop_words]
# # filter out short tokens
# tokens = [word for word in tokens if len(word) > 1]
# print(tokens)


fried
(3602, 3612)


In [3]:
import pandas as pd
from sklearn import svm
import numpy as np

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report
# from sklearn.metrics import accuracy_score
import pickle
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
print('success')
'hello world'

'''Load data set'''
df1 = pd.read_csv("weightxy_data2.csv")
print(df1.shape)
print(df1.iloc[:,:-1].shape)
'hello world'



success
(3602, 3613)
(3602, 3612)


'hello world'

In [4]:
# print(df1['Unnamed: 0'])

count = 0
# processed_df1 = df1[['class_']].copy()
processed_df1 = pd.DataFrame()
temp_one_AT = pd.DataFrame()
temp_df2 = pd.DataFrame()

for column in df1:
#     print(column)
    temp_array = np.array(df1[column].values)
#     print(np.where(temp_array > 0)[0].shape)
    if np.where(temp_array > 0)[0].shape[0] <= 1:
        if pd.DataFrame.max(df1[column]) > 1:
            temp_one_AT[column] = df1[[column]].copy()
            count = count + 1
            pass
        else:
            temp_df2[column] = df1[[column]].copy()
            processed_df1[column] = df1[[column]].copy()
#             count = count + 1
#             print(column)
    else: 
        processed_df1[column] = df1[[column]].copy()
        pass

processed_df1['temp_1'] = temp_one_AT.sum(axis=1)
# processed_df1[temp_2] = temp_df2.sum(axis=1)
    
print(count)
print('df1', df1.shape)
print('processed df1', processed_df1.shape)
#     if pd.DataFrame.max(df1[column]) == 0:
#         print(column)

111
df1 (3602, 3613)
processed df1 (3602, 3503)


In [1]:
##
import numpy as np
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

X, Y = df1.drop(['class_'], axis = 1), df1['class_']
# X, Y = processed_df1.drop(['class_'], axis = 1), processed_df1['class_']

# encode class values as integers
encoder = LabelEncoder()
# print(encoder.classes_)

encoder.fit(Y)
print(encoder.classes_)
print(Y[0])
encoded_Y = encoder.transform(Y)
# print(encoded_Y.classes)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y)
print('Working on model...')

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# model = Sequential()

# create model
# model = Sequential()
# model.add(Dense(8, input_dim=3612, activation='relu'))
# model.add(Dense(3, activation='softmax'))
# Compile model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Start
''' Trying out CV'''
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
# results = cross_val_score(model, X, Y, cv=kfold)
# y_pred_ma = cross_val_predict(model, X, Y, cv=kfold)
y_pred_ma = cross_validate(model, X, Y, cv=kfold)
# end

# X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size = 0.2, random_state = 42)
# print(np.arange(Y_test).reshape())

# y_pred = np.zeros_like(Y_test)

# model.fit(X_train, Y_train, epochs = 10, batch_size = 10, shuffle = False)

# pred = model.predict(X_test)
# print(pred)
    


# for pred_index, pred_val in enumerate(y_pred):

#     if y_pred[pred_index][0]:
#         y_pred_label[pred_index] = -1
#     elif y_pred[pred_index][1]:
#         y_pred_label[pred_index] = 0    
#     else:
#         y_pred_label[pred_index] = 1    

        
    
# from sklearn.metrics import accuracy_score, classification_report
# mod_accracy = accuracy_score(Y_test, y_pred)
# print(mod_accuracy)




Using TensorFlow backend.


KeyboardInterrupt: 

In [9]:
# print(results.mean())

In [10]:
# y_pred_ma = cross_val_predict(model, X, Y, cv=kfold)

In [11]:
mod_accracy = accuracy_score(y_pred_ma, Y)
print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))

target_names = ['0', '1', '-1']
print('Pr, Re scores wrt each class')
print(classification_report(y_pred_ma, Y, target_names=target_names))



Model Accuracy is: 74.4586%
Pr, Re scores wrt each class
             precision    recall  f1-score   support

          0       0.62      0.64      0.63       780
          1       0.53      0.53      0.53       632
         -1       0.86      0.85      0.85      2190

avg / total       0.75      0.74      0.75      3602

